In [ ]:
#Faz 1: Hazırlık ve Veri Seti

In [ ]:
#Veriyi Anlama (Keşifsel Veri Analizi - EDA):

In [ ]:
import pandas as pd

# Bazen bu dosyada standart olmayan karakterler olabiliyor, 
# bu yüzden 'encoding' parametresini eklemek hataları önleyebilir. bu kod ile satır ve sütunların ne anlama geldiğini anladık.
try:
    column_descriptions = pd.read_csv('HomeCredit_columns_description.csv', encoding='utf-8')
except UnicodeDecodeError:
    column_descriptions = pd.read_csv('HomeCredit_columns_description.csv', encoding='latin1')


# Pandas'ın normalde tüm satırları göstermeme limitini kaldıralım ki sözlüğümüzün tamamını görelim.
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None) # Sütun içeriğinin tamamını göster

print("Sütun Açıklamaları Dosyasının İçeriği:")
display(column_descriptions)

# Ayarları daha sonra normale döndürmek istersen:
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_colwidth')

In [ ]:
import pandas as pd

# Pandas'ın tüm sütunları göstermesini sağlayalım
pd.set_option('display.max_columns', None)

print("Ana veri seti (application_train.csv) yükleniyor...")
# Bu dosya büyük olduğu için yüklenmesi birkaç saniye sürebilir.

# Ana eğitim verimizi bir pandas DataFrame'e yüklüyoruz.
# Bu yapı, veriyi tablo gibi işlememizi sağlar.
df_train = pd.read_csv('application_train.csv')

print("Yükleme tamamlandı!")
print("-" * 50)

# 1. Verinin Boyutlarına Bakalım
# .shape bize (satır sayısı, sütun sayısı)'nı verir.
print(f"Veri setinin boyutu (Satır Sayısı, Sütun Sayısı): {df_train.shape}")
print("-" * 50)


# 2. Verinin İlk 5 Satırına Göz Atalım
# .head() metodu, verinin genel yapısını anlamak için harikadır.
print("Veri setinin ilk 5 satırı:")
display(df_train.head())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Hedef değişkenin dağılımını analiz edelim
print("Hedef Değişken (TARGET) Dağılımı:")
target_counts = df_train['TARGET'].value_counts()
print(target_counts)

print("\nYüzdesel Dağılım:")
target_percentages = df_train['TARGET'].value_counts(normalize=True) * 100
print(target_percentages)

# Bu dağılımı görselleştirelim
plt.figure(figsize=(8, 6))
sns.countplot(x='TARGET', data=df_train)
plt.title('Kredi Geri Ödeme Durumu Dağılımı\n (0: Sorunsuz Ödeme, 1: Ödeme Güçlüğü)', fontsize=14)
plt.xlabel('Hedef Değişken (TARGET)', fontsize=12)
plt.ylabel('Müşteri Sayısı', fontsize=12)

# Grafiği kaydet (opsiyonel)
plt.savefig('hedef_degisken_dagilimi.png')

plt.show()

In [ ]:
# Her sütundaki eksik değerlerin toplamını hesapla
missing_values = df_train.isnull().sum()

# Eksik değerlerin yüzdesini hesapla
missing_percentage = (missing_values / len(df_train)) * 100

# İkisini birleştirip bir DataFrame oluştur
missing_info = pd.DataFrame({'Eksik Değer Sayısı': missing_values, 'Yüzde (%)': missing_percentage})

# Sadece eksik veri içeren sütunları göster ve yüzdesine göre büyükten küçüğe sırala
missing_info_sorted = missing_info[missing_info['Eksik Değer Sayısı'] > 0].sort_values(by='Yüzde (%)', ascending=False)

print("Eksik Veri İçeren Sütunlar (Yüzdeye Göre Sıralı):")
display(missing_info_sorted)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))

# TARGET=1 (ödeme güçlüğü çekenler) için kredi miktarının dağılımı
sns.kdeplot(df_train.loc[df_train['TARGET'] == 1, 'AMT_CREDIT'], label='Target == 1', color='red')

# TARGET=0 (sorunsuz ödeyenler) için kredi miktarının dağılımı
sns.kdeplot(df_train.loc[df_train['TARGET'] == 0, 'AMT_CREDIT'], label='Target == 0', color='green')

plt.legend()
plt.xlabel('Kredi Miktarı (AMT_CREDIT)', fontsize=12)
plt.ylabel('Yoğunluk', fontsize=12)
plt.title('Kredi Miktarının Müşteri Risk Durumuna Göre Dağılımı', fontsize=14)
plt.show()

In [ ]:
#Grafiğin Yorumlanması
#Benzer Dağılımlar: İlk bakışta en çok göze çarpan şey, iki grubun dağılımının (kırmızı ve yeşil çizgiler) genel olarak birbirine oldukça benzemesidir.
#Her iki grup da en çok düşük miktarlı krediler çekiyor ve kredi miktarı arttıkça o krediyi çeken müşteri sayısı azalıyor (sağa doğru uzun bir kuyruk var).
#İnce Bir Fark: Ancak biraz daha dikkatli bakarsak, kırmızı çizginin (riskli müşteriler, TARGET=1) tepe noktasının, yeşil çizgiye (risksiz müşteriler, TARGET=0) göre çok az daha solda olduğunu görebiliriz. 
#Bu, ödeme güçlüğü çekenler arasında çok düşük miktarlı kredi çekenlerin oranının, sorunsuz ödeyenlere göre bir tık daha fazla olabileceğini ima ediyor.
#En Önemli Çıkarım: Bu iki dağılım arasında keskin ve net bir ayrım yok. Bu bize şunu söylüyor: Sadece AMT_CREDIT (kredi miktarı) sütununa bakarak bir müşterinin riskli olup olmadığına kesin olarak karar vermek çok zor. 
#Demek ki modelimizin doğru karar verebilmesi için bu özellik tek başına yeterli olmayacak; birçok farklı özelliği bir arada değerlendirmesi gerekecek. İşte bu yüzden 122 tane özelliğimiz var!

#

In [ ]:
#Faz 2: Model Geliştirme


In [ ]:
#Veri Ön İşleme (Preprocessing)

In [ ]:
#Bu fazda yapacaklarımız:

#Sütunları Temizleme: %50'den fazla eksik veriye sahip olan ve modelimize faydası olmayacak sütunları veri setinden çıkaracağız.
#Kategorik Verileri Sayısallaştırma: NAME_CONTRACT_TYPE (Cash loans/Revolving loans) veya CODE_GENDER (M/F) gibi metin içeren sütunları, 
#modellerin anlayabileceği sayılara (0, 1 gibi) dönüştüreceğiz. Buna "Encoding" diyoruz..
#Kalan Eksik Verileri Doldurma: Orta ve az seviyede eksik veriye sahip sütunlardaki boşlukları, 
#ortalama, medyan veya "Bilinmiyor" gibi stratejilerle doldurulacak (Imputation).

In [ ]:
# Başlamadan önce veri setimizin ilk boyutunu hatırlayalım
print(f"Temizlik öncesi veri seti boyutu (Satır, Sütun): {df_train.shape}")
print("-" * 50)

# Her sütundaki eksik değerlerin oranını (yüzdesini değil, 0-1 arası) hesaplayalım
missing_ratios = df_train.isnull().sum() / len(df_train)

# Bu oranların %50'den (yani 0.5'ten) büyük olduğu sütunları bulalım
columns_to_drop = missing_ratios[missing_ratios > 0.5].index

print(f"Silinecek sütun sayısı: {len(columns_to_drop)}")
print("Bu sütunlar şunlar:")
# print(columns_to_drop.tolist()) # Eğer silinecek sütunların listesini görmek istersen bu satırı aktif et

# Belirlenen sütunları 'axis=1' (sütun bazında) diyerek veri setinden silelim
df_train_cleaned = df_train.drop(columns = columns_to_drop)

# Temizlik sonrası yeni boyutu kontrol edelim
print("-" * 50)
print(f"Temizlik sonrası YENİ veri seti boyutu: {df_train_cleaned.shape}")
print("\nİşlem başarılı! Artık daha temiz bir veri setine sahibiz.")

In [ ]:
# Bir önceki adımda oluşturduğumuz df_train_cleaned'i kullanalım
# Önce kategorik ve sayısal sütunları ayıralım
# Not: TARGET sütunu hedefimiz olduğu için onu özelliklerden ayrı tutuyoruz.
df_processed = df_train_cleaned.drop(columns=['TARGET'])
y = df_train_cleaned['TARGET'] # Hedef değişkeni y'ye atadık

# Kategorik sütunları bulalım
categorical_cols = df_processed.select_dtypes(include=['object']).columns

# One-Hot Encoding işlemini yapalım
# Bu işlem metin içeren sütunları 0'lar ve 1'lerden oluşan yeni sütunlara çevirir
df_processed_encoded = pd.get_dummies(df_processed, columns=categorical_cols, dummy_na=False) # dummy_na=False ile NaN'lar için ayrı sütun oluşturmuyoruz

print(f"One-Hot Encoding öncesi sütun sayısı: {df_processed.shape[1]}")
print(f"One-Hot Encoding sonrası sütun sayısı: {df_processed_encoded.shape[1]}")
print("-" * 50)

# Şimdi, tüm veri seti sayısal hale geldi.
# Kalan eksik değerleri (NaN) medyan ile dolduralım.
# scikit-learn kütüphanesinden SimpleImputer'ı kullanalım
from sklearn.impute import SimpleImputer

# Medyan stratejisi, aykırı değerlerden daha az etkilendiği için genellikle daha güvenlidir.
imputer = SimpleImputer(strategy='median')

# Imputer'ı eğitelim ve veriyi dönüştürelim
df_final = pd.DataFrame(imputer.fit_transform(df_processed_encoded), columns=df_processed_encoded.columns)

print("Eksik verileri doldurduktan sonraki durum:")
# Kalan toplam eksik veri sayısını kontrol edelim
print(f"Son veri setindeki toplam eksik veri sayısı: {df_final.isnull().sum().sum()}")
print(f"Nihai veri setimizin boyutu (Satır, Sütun): {df_final.shape}")
print("\nTebrikler! Veri setimiz artık modelleme için tamamen temiz ve hazır!")

In [ ]:
#80 özellik sütunumuz vardı.
#Metin içeren kategorik sütunları One-Hot Encoding ile sayısala çevirdik ve bu işlem sütun sayımızı 193'e çıkardı.
#Kalan tüm eksik değerleri medyan ile doldurduk ve veri setimizde 0 eksik değer kaldı.



In [ ]:
#Faz 3 - Model Geliştirme

In [ ]:
!pip install xgboost

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

# 1. Veriyi Eğitim ve Test Setlerine Ayırma
# X -> Özelliklerimiz (df_final)
# y -> Hedefimiz (TARGET)
# test_size=0.2 -> Verinin %20'sini test için ayır
# stratify=y -> Ayırma işlemi yapılırken eğitim ve test setlerindeki %8'lik TARGET oranını koru. Bu çok önemli!
# random_state=42 -> Her seferinde aynı ayırma işleminin yapılmasını sağlar.
X_train, X_test, y_train, y_test = train_test_split(df_final, y, test_size=0.2, random_state=42, stratify=y)

print("Veri setleri ayrıldı:")
print(f"Eğitim seti boyutu: {X_train.shape}")
print(f"Test seti boyutu: {X_test.shape}")
print("-" * 50)

# 2. Dengesiz veri seti için 'scale_pos_weight' parametresini hesaplama
# Bu parametre, azınlık sınıfına (TARGET=1) daha fazla önem verilmesini sağlar.
scale_pos_weight = y_train.value_counts()[0] / y_train.value_counts()[1]
print(f"Dengesizlik oranı (scale_pos_weight): {scale_pos_weight:.2f}")
print("-" * 50)


# 3. XGBoost Modelini Tanımlama ve Eğitme
print("XGBoost modeli eğitiliyor...")

# Modeli, hesapladığımız parametre ile tanımlıyoruz.
# n_estimators, modelin kaç tane ağaç kuracağını belirtir. İlk deneme için 100 iyi bir başlangıç.
# max_depth, her ağacın ne kadar derin olabileceğini kontrol eder. Overfitting'i önlemeye yardımcı olur.
model = XGBClassifier(n_estimators=200, max_depth=5, scale_pos_weight=scale_pos_weight, random_state=42, n_jobs=-1)

# Modeli eğitim verileriyle eğitiyoruz. Bu işlem birkaç dakika sürebilir.
model.fit(X_train, y_train)

print("Model başarıyla eğitildi!")

In [ ]:
# Değerlendirme aşaması

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Test verisi üzerinde tahmin yapma
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1] # ROC AUC için olasılık değerleri gerekir

# 2. Performans metriklerini hesaplama
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print("--- Model Performans Metrikleri ---")
print(f"Doğruluk (Accuracy): {accuracy:.4f}")
print(f"Kesinlik (Precision): {precision:.4f}")
print(f"Duyarlılık (Recall): {recall:.4f}")
print(f"ROC AUC Skoru: {roc_auc:.4f}")
print("-" * 35)

# 3. Karmaşıklık Matrisini (Confusion Matrix) oluşturma ve görselleştirme
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Tahmin: Risksiz (0)', 'Tahmin: Riskli (1)'],
            yticklabels=['Gerçek: Risksiz (0)', 'Gerçek: Riskli (1)'])
plt.ylabel('Gerçek Değer')
plt.xlabel('Tahmin Edilen Değer')
plt.title('Karmaşıklık Matrisi (Confusion Matrix)', fontsize=15)
plt.show()

# 4. Sınıflandırma Raporu
print("\n--- Sınıflandırma Raporu ---")
print(classification_report(y_test, y_pred, target_names=['Risksiz (0)', 'Riskli (1)']))

In [ ]:
# Duyarlılık (Recall): Gerçekten riskli olan müşterilerin yüzde kaçını doğru tespit edebildiğimizi gösterir. Yüksek olması, batık kredi riskini kaçırmadığımız anlamına gelir ve genellikle bu tür problemler için en önemli metriklerden biridir.

# Kesinlik (Precision): "Riskli" olarak tahmin ettiklerimizin yüzde kaçının gerçekten riskli olduğunu gösterir. Yüksek olması, sağlıklı müşterileri yanlışlıkla "riskli" olarak etiketleme oranımızın düşük olduğu anlamına gelir.

# Doğruluk (Accuracy): Tüm tahminlerin (riskli/risksiz) genel olarak yüzde kaçının doğru olduğunu gösterir. Dengesiz veri setlerinde (bizimki gibi) yanıltıcı olabileceği için tek başına güvenilmez.

# ROC AUC Skoru: Modelin, riskli ve risksiz müşteri profillerini birbirinden genel olarak ne kadar iyi ayırt edebildiğini ölçer. 1'e ne kadar yakınsa, modelin ayırt etme gücü o kadar iyidir (0.5 rastgele tahmindir).

In [ ]:
!pip install shap

In [ ]:
import shap

# SHAP'ın JavaScript görsellerini not defterinde düzgün göstermesi için bu satırı çalıştıralım
shap.initjs()

print("SHAP değerleri hesaplanıyor. Bu işlem birkaç dakika sürebilir...")

# Modelimizin bir ağaç tabanlı model (XGBoost) olduğunu belirten bir "Açıklayıcı" (Explainer) oluşturuyoruz.
explainer = shap.TreeExplainer(model)

# Test setimiz üzerinden SHAP değerlerini hesaplıyoruz.
# Bu işlem tüm test seti için uzun sürebileceğinden, genellikle daha hızlı sonuç almak için bir alt küme kullanılır.
# Ancak şimdilik tüm test setiyle deneyelim.
shap_values = explainer.shap_values(X_test)

print("Hesaplama tamamlandı. Özet grafiği oluşturuluyor...")

# Modelimiz için en önemli özellikleri gösteren özet grafiği (summary plot)
# Bu grafik, hangi özelliğin tahminleri ne yönde ve ne kadar etkilediğini gösterir.
shap.summary_plot(shap_values, X_test, plot_type="dot")

In [ ]:
import shap

# Daha önce hesapladığımız explainer ve shap_values'ı kullanacağız

# 1. Başarıyla "Riskli" olarak tahmin edilen bir müşteri bulalım
# Gerçek değeri 1 ve tahmin edilen değeri 1 olan ilk müşterinin index'ini alalım
true_positive_customer_index = X_test[(y_test == 1) & (y_pred == 1)].index[0]
# Bu index'in test setindeki yerini bulalım
true_positive_loc = X_test.index.get_loc(true_positive_customer_index)

print(f"--- REDDEDİLEN Müşteri Analizi (Index: {true_positive_customer_index}) ---")
print("Bu grafikte kırmızı oklar riski ARTIRAN, mavi oklar ise riski AZALTAN faktörleri gösterir.")
# Bu müşteri için Force Plot'u oluşturalım
shap.force_plot(explainer.expected_value, shap_values[true_positive_loc,:], X_test.loc[true_positive_customer_index])

In [ ]:
# 2. Başarıyla "Risksiz" olarak tahmin edilen bir müşteri bulalım
# Gerçek değeri 0 ve tahmin edilen değeri 0 olan ilk müşterinin index'ini alalım
true_negative_customer_index = X_test[(y_test == 0) & (y_pred == 0)].index[0]
# Bu index'in test setindeki yerini bulalım
true_negative_loc = X_test.index.get_loc(true_negative_customer_index)

print(f"\n\n--- ONAYLANAN Müşteri Analizi (Index: {true_negative_customer_index}) ---")
print("Bu grafikte kırmızı oklar riski ARTIRAN, mavi oklar ise riski AZALTAN faktörleri gösterir.")
# Bu müşteri için Force Plot'u oluşturalım
shap.force_plot(explainer.expected_value, shap_values[true_negative_loc,:], X_test.loc[true_negative_customer_index])